In [1]:
import sys,os
__script_path=os.path.abspath(globals().get('__file__','.'))
__script_dir = os.path.dirname(__script_path)
root_dir = os.path.abspath(f'{__script_dir}/../../../..')
print(root_dir)
for lib in [root_dir][::-1]:
    if lib in sys.path:
        sys.path.remove(lib)
    sys.path.insert(0,lib)

from libs.common import *
from config.config import *

c:\Users\Admin\Data\ads_fraud_detection


c:/Users/Admin/Data/ads_fraud_detection


# 3. Tiền xử lí dữ liệu

In [ ]:
save_dir=f"{exps_dir}/exp4/exp_gau"

save_dir = f"{save_dir}"
os.makedirs(save_dir, exist_ok=True)
test_size=0.33
seed=42

In [3]:
test_size=0.33
seed=42
data=pd.read_csv(f'{exps_dir}/exp4/data_EDA.csv')
data

,user_id,signup_time,purchase_time,purchase_value,device_id,source,browser,sex,age,ip_address,class,IP_country,freq_device
0,22058,2015-02-24 22:55:49,2015-04-18 02:47:11,34,QVPSPJUOCKZAR,SEO,Chrome,M,39,7.327584e+08,0,Japan,1
1,333320,2015-06-07 20:39:50,2015-06-08 01:38:54,16,EOGFQPIZPYXFZ,Ads,Chrome,F,53,3.503114e+08,0,United States,1
2,1359,2015-01-01 18:52:44,2015-01-01 18:52:45,15,YSSKYOSJHPPLJ,SEO,Opera,M,53,2.621474e+09,1,United States,12
3,48288,2015-01-01 18:52:41,2015-01-01 18:52:42,15,YSSKYOSJHPPLJ,SEO,Opera,M,53,2.621474e+09,1,United States,12
4,128706,2015-01-01 18:52:40,2015-01-01 18:52:41,15,YSSKYOSJHPPLJ,SEO,Opera,M,53,2.621474e+09,1,United States,12
...,...,...,...,...,...,...,...,...,...,...,...,...,...
151107,356404,2015-02-04 10:27:49,2015-05-07 19:37:56,41,FXBMCMIOLLNCJ,SEO,Chrome,M,26,1.827348e+09,0,United States,1
151108,151643,2015-02-15 20:03:27,2015-05-07 19:28:52,23,THQOCEAVXXAIW,Ads,FireFox,M,18,1.691181e+09,0,United States,1
151109,274471,2015-05-15 17:43:29,2015-05-26 12:24:39,35,LYSFABUCPCGBA,SEO,Safari,M,32,2.439047e+09,0,Netherlands,1
151110,368416,2015-03-03 23:07:31,2015-05-20 07:07:47,40,MEQHCSJUBRBFE,SEO,IE,F,26,2.748471e+09,0,Japan,1


In [4]:
# outliers_list=dict(np.load(f'{save_dir}/outliers.npz'))['outliers']
# def remove_outliers_iqr(data, column):
#     Q1 = data[column].quantile(0.25)
#     Q3 = data[column].quantile(0.75)
#     IQR = Q3 - Q1
#     lower_bound = Q1 - 1.5 * IQR
#     upper_bound = Q3 + 1.5 * IQR
#     return data[(data[column] >= lower_bound) & (data[column] <= upper_bound)]

# # Áp dụng hàm cho các cột chứa dữ liệu cần loại bỏ ngoại lệ
# for column in outliers_list:
#     data_train = remove_outliers_iqr(data_train, column)
# data_train

In [5]:
data.isnull().sum()

user_id           0
signup_time       0
purchase_time     0
purchase_value    0
device_id         0
source            0
browser           0
sex               0
age               0
ip_address        0
class             0
IP_country        0
freq_device       0
dtype: int64

In [6]:
data.head()

,user_id,signup_time,purchase_time,purchase_value,device_id,source,browser,sex,age,ip_address,class,IP_country,freq_device
0,22058,2015-02-24 22:55:49,2015-04-18 02:47:11,34,QVPSPJUOCKZAR,SEO,Chrome,M,39,7.327584e+08,0,Japan,1
1,333320,2015-06-07 20:39:50,2015-06-08 01:38:54,16,EOGFQPIZPYXFZ,Ads,Chrome,F,53,3.503114e+08,0,United States,1
2,1359,2015-01-01 18:52:44,2015-01-01 18:52:45,15,YSSKYOSJHPPLJ,SEO,Opera,M,53,2.621474e+09,1,United States,12
3,48288,2015-01-01 18:52:41,2015-01-01 18:52:42,15,YSSKYOSJHPPLJ,SEO,Opera,M,53,2.621474e+09,1,United States,12
4,128706,2015-01-01 18:52:40,2015-01-01 18:52:41,15,YSSKYOSJHPPLJ,SEO,Opera,M,53,2.621474e+09,1,United States,12


In [7]:
# # Since we know the same machine_id can be used for several transaction we will 
# # look at how many times on average devices with duplicates are used and their distribution

# # data = pd.read_csv(f"{exps_dir}/exp4/Fraud_data_with_country.csv")

# device_duplicates = pd.DataFrame(data.groupby(by = "device_id").device_id.count())  # at this moment, index column name and first column name both are equal to "device_id"
# device_duplicates.rename(columns={"device_id": "freq_device"}, inplace=True)           # hence we need to replace the "device_id" column name
# device_duplicates.reset_index(level=0, inplace= True)                                  # and then we turn device_id from index to column

# dupli = device_duplicates[device_duplicates.freq_device >1]
# # print("On average, when a device is used more than once it is used {mean} times, and the most used machine was used {maxi} times"
# #       .format(mean = int(dupli.mean()*10)/10, maxi = int(dupli.freq_device.max()*10)/10))

# dupli = device_duplicates[device_duplicates.freq_device >2]
# # print("On average, when a device is used more than twice it is used {mean} times"
# #       .format(mean = int(dupli.mean()*10)/10, maxi = int(dupli.freq_device.max()*10)/10))

# # finally we merge with data
# data = data.merge(device_duplicates, on= "device_id")


In [8]:
data

,user_id,signup_time,purchase_time,purchase_value,device_id,source,browser,sex,age,ip_address,class,IP_country,freq_device
0,22058,2015-02-24 22:55:49,2015-04-18 02:47:11,34,QVPSPJUOCKZAR,SEO,Chrome,M,39,7.327584e+08,0,Japan,1
1,333320,2015-06-07 20:39:50,2015-06-08 01:38:54,16,EOGFQPIZPYXFZ,Ads,Chrome,F,53,3.503114e+08,0,United States,1
2,1359,2015-01-01 18:52:44,2015-01-01 18:52:45,15,YSSKYOSJHPPLJ,SEO,Opera,M,53,2.621474e+09,1,United States,12
3,48288,2015-01-01 18:52:41,2015-01-01 18:52:42,15,YSSKYOSJHPPLJ,SEO,Opera,M,53,2.621474e+09,1,United States,12
4,128706,2015-01-01 18:52:40,2015-01-01 18:52:41,15,YSSKYOSJHPPLJ,SEO,Opera,M,53,2.621474e+09,1,United States,12
...,...,...,...,...,...,...,...,...,...,...,...,...,...
151107,356404,2015-02-04 10:27:49,2015-05-07 19:37:56,41,FXBMCMIOLLNCJ,SEO,Chrome,M,26,1.827348e+09,0,United States,1
151108,151643,2015-02-15 20:03:27,2015-05-07 19:28:52,23,THQOCEAVXXAIW,Ads,FireFox,M,18,1.691181e+09,0,United States,1
151109,274471,2015-05-15 17:43:29,2015-05-26 12:24:39,35,LYSFABUCPCGBA,SEO,Safari,M,32,2.439047e+09,0,Netherlands,1
151110,368416,2015-03-03 23:07:31,2015-05-20 07:07:47,40,MEQHCSJUBRBFE,SEO,IE,F,26,2.748471e+09,0,Japan,1


In [9]:
import calendar

# data = pd.read_csv("../input/datas-fraud/Fraud_data_with_country.csv")
# data = data.merge(device_duplicates, on= "device_id")

# --- 1 ---
# Categorisation column freq_device
# We see a clear correlation between freq_device and fraudulent activities. We are going to split freq_device in 7 categories
data.freq_device = pd.to_numeric(data.freq_device, errors='coerce')

data.freq_device = data.freq_device.apply(lambda x :
                                                str(x) if x <5 else
                                                "5-10" if x>=5 and x<=10 else
                                                "11-15" if x>10 and x<=15 else
                                                "> 15")

# We convert signup_time and purchase_time en datetime

# data.signup_time = pd.to_datetime(data.signup_time, format = '%d-%m-%Y %H:%M:%S')
# data.purchase_time = pd.to_datetime(data.purchase_time, format = '%d-%m-%Y %H:%M:%S')
# # Convert signup_time and purchase_time to datetime with the correct format
data.signup_time = pd.to_datetime(data.signup_time, format='%Y-%m-%d %H:%M:%S')
data.purchase_time = pd.to_datetime(data.purchase_time, errors='coerce')

# We add features 

# --- 2 ---
# Column month
data["month_purchase"] = data.purchase_time.apply(lambda x: calendar.month_name[x.month])

# --- 3 ---
# Column week
data["weekday_purchase"] = data.purchase_time.apply(lambda x: calendar.day_name[x.weekday()])

# --- 4 ---
# Column hour_of_the_day
data["hour_of_the_day"] = data.purchase_time.apply(lambda x: x.hour)

# --- 5 ---
# Column seconds_since_signup
data["seconds_since_signup"]= (data.purchase_time - data.signup_time).apply(lambda x : x.total_seconds())

# --- 6 ---
# Column countries_from_device (ie. number of different countries per device_id)

# We flag devices that committed purchases from different countries
# First we groupby device_id and IP_country which will give us a DF with a sublist of country for each device_id
country_count = data.groupby(by =[ "device_id","IP_country"]).count().reset_index()

# Then we can count the number of different countries by device_id
country_count = pd.DataFrame(country_count.groupby(by="device_id").count().IP_country)

# Finally we can merge this to our main data
data = data.merge(country_count, left_on="device_id", right_index=True)
data.rename(columns={"IP_country_x": "IP_country" , "IP_country_y":"countries_from_device"}, inplace = True)

# --- 7 ---
# Column risk_country which classifies each country based on historical fraud rate from these countries



risk_country = data.groupby(by="IP_country")['class'].mean().sort_values(ascending=False).to_frame()

# We classify each country between Very High risk, High risk, Medium risk and low risk
risk_country["risk_country"] = risk_country["class"].apply(lambda x : 
                                                           "Very High risk" if x > 0.25 else
                                                           "High risk" if x > 0.05 else
                                                           "Medium risk" if x > 0.01 else
                                                           "Low risk")
                                                
# We merge with data
data = data.merge(risk_country.drop("class", axis = 1), left_on= "IP_country", right_index = True)

# --- 8 ---
# Column "quick_purchase" : categorise time between sign_up and purchase
data["quick_purchase"] = data.seconds_since_signup.apply(lambda x : 1 if x < 30 else 0)

# --- 9 ---
# Column freq_same_purchase : indicates how many times a given device_id purchased an item of the same value

# We generate a table indicating for each line if the couple device_id / purchase_value has duplicates
duplicate = data.duplicated(subset = ["purchase_value","device_id"], keep = False)
# We merge it with a DF containing purchase_value and device_id
duplicate = pd.concat([data.loc[:,["purchase_value","device_id"]], duplicate],axis=1)
# We build a DataFrame which gives us the number of duplicates for each combination of device_id / purchase_value
duplicate = duplicate.groupby(by = ["device_id","purchase_value"]).sum()
# We categorise number of time the same purchase has been made 
duplicate["freq_same_purchase"] = duplicate[0].apply(lambda x : 
                                                x if x < 5 else
                                                "5-10" if x<=10 else
                                                "11-15" if x<= 15 else
                                                ">15" 
                                               )
                                            
# We merge the result with main data            
data.merge(duplicate.drop(0,axis=1), left_on=["device_id","purchase_value"], right_index = True)

# --- 10 ---- 
# age categorisation

data["age_category"] = data.age.apply(lambda x : 
                 "< 40" if x < 40 else
                 "40 - 49" if x < 50 else
                 "50 -59" if x < 60 else
                 "60 - 69" if x < 70 else
                 " > 70")

# ---- 11 ----
# Hour of the day categorisation
data["period_of_the_day"] = data.hour_of_the_day.apply(lambda x : 
                                                             "late night" if x < 4 else
                                                             "early morning" if x < 8 else
                                                             "morning" if x < 12 else
                                                             "early arvo" if x < 16 else
                                                             "arvo" if x < 20 else
                                                             "evening"
                                                            )

# ---- 12 -----
# First_purchase 

# Assuming a potential scammer would want to check out the website before committing any fraudulent activity
# we want to know if for a given device_ID if the row corresponds to a first operation or not

# However the following code take a long time to run! That's why we have save the resulting Series in the file 
# Series_first_purchase.csv to prevent waiting time when we reset the kernel

# Original code :
data["first_purchase"] = data.apply(lambda x : 
                                         1 if x.purchase_time == data.purchase_time[data.device_id == x.device_id].min() else 0,
                                         axis =1)
#data.first_purchase.to_csv("datas/Series_first_purchase.csv",index =False)

# data["first_purchase"] = pd.read_csv("../input/datas-fraud/Series_first_purchase.csv", )


# data.to_csv("../data_with_first_feature_eng.csv")

In [10]:
data.weekday_purchase

0          Saturday
61         Thursday
70           Monday
84           Monday
91           Sunday
            ...    
127261     Saturday
132261      Tuesday
135452    Wednesday
137201     Thursday
148104      Tuesday
Name: weekday_purchase, Length: 151112, dtype: object

In [11]:
data["is_january"] = data.weekday_purchase.apply(lambda x : 1 if x == "January" else 0)


In [12]:
recycle_ds = pd.DataFrame()
def to_recycle_bin(column) :
    recycle_ds[column] = data[column]
    data.drop(column, axis=1, inplace = True)
    
# Function to get a column back from recycle_ds

def restore_from_bin(column) :
    data[column] = data[column]
    recycle_ds.drop(column, axis=1, inplace = True)


In [13]:
# We get rid of useless columns

for colonne in ["user_id",
                "signup_time",
                "purchase_time",
                "device_id",
                "ip_address",
                "IP_country",
                "hour_of_the_day",
               "seconds_since_signup",
               "age"] :
    to_recycle_bin(colonne)
    
data.dropna(inplace = True)

# 4.Chuẩn hóa dữ liệu

In [14]:
numeric_columns=dict(np.load(f'{exps_dir}/exp4/columns_dtype.npz',allow_pickle=True))['numeric_columns']
numeric_columns

array(['user_id', 'purchase_value', 'age', 'ip_address', 'class',
       'freq_device'], dtype='<U14')

In [15]:
category_columns=dict(np.load(f'{exps_dir}/exp4/columns_dtype.npz',allow_pickle=True))['category_columns']
category_columns

array(['signup_time', 'purchase_time', 'device_id', 'source', 'browser',
       'sex', 'IP_country'], dtype='<U13')

In [16]:
# Determine numeric and object columns
numeric_columns = data.select_dtypes(include=['number']).columns.tolist()
category_columns = data.select_dtypes(include=['object']).columns.tolist()
numeric_columns.remove('class')
print("Numeric columns:", numeric_columns)
print("Object columns:", category_columns)

Numeric columns: ['purchase_value', 'countries_from_device', 'quick_purchase', 'first_purchase', 'is_january']
Object columns: ['source', 'browser', 'sex', 'freq_device', 'month_purchase', 'weekday_purchase', 'risk_country', 'age_category', 'period_of_the_day']


In [17]:
data_train, data_test = train_test_split(data, test_size = 0.33, random_state=seed)
print(data_train.shape,data_test.shape)
data_train.to_csv(f'{save_dir}/datatrain.csv', index=None)
data_test.to_csv(f'{save_dir}/datatest.csv', index=None)

(101245, 15) (49867, 15)


In [18]:
data_train

,purchase_value,source,browser,sex,class,freq_device,month_purchase,weekday_purchase,countries_from_device,risk_country,quick_purchase,age_category,period_of_the_day,first_purchase,is_january
151060,21,Direct,Safari,F,0,1,November,Sunday,1,High risk,0,< 40,evening,1,0
119520,50,Direct,Chrome,M,0,1,September,Wednesday,1,High risk,0,40 - 49,late night,1,0
45813,49,Ads,Chrome,F,0,1,March,Sunday,1,High risk,0,< 40,morning,1,0
34353,50,SEO,FireFox,M,1,11-15,January,Thursday,1,High risk,1,40 - 49,late night,0,0
61518,54,SEO,FireFox,F,0,1,June,Sunday,1,High risk,0,< 40,early arvo,1,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
51799,41,SEO,IE,M,0,2,June,Saturday,2,High risk,0,< 40,early morning,1,0
69108,13,Ads,FireFox,M,0,1,January,Wednesday,1,High risk,0,< 40,early arvo,1,0
131577,54,SEO,IE,F,0,1,October,Wednesday,1,High risk,0,< 40,evening,1,0
140342,57,SEO,FireFox,M,0,1,February,Tuesday,1,High risk,0,50 -59,arvo,1,0


(101245, 56) (49867, 56)
(101245, 2) (49867, 2)


In [20]:
X_train

,purchase_value,countries_from_device,quick_purchase,first_purchase,is_january,source_Ads,source_Direct,source_SEO,browser_Chrome,browser_FireFox,browser_IE,browser_Opera,browser_Safari,sex_F,sex_M,freq_device_1,freq_device_11-15,freq_device_2,freq_device_3,freq_device_4,freq_device_5-10,freq_device_> 15,month_purchase_April,month_purchase_August,month_purchase_December,month_purchase_February,month_purchase_January,month_purchase_July,month_purchase_June,month_purchase_March,month_purchase_May,month_purchase_November,month_purchase_October,month_purchase_September,weekday_purchase_Friday,weekday_purchase_Monday,weekday_purchase_Saturday,weekday_purchase_Sunday,weekday_purchase_Thursday,weekday_purchase_Tuesday,weekday_purchase_Wednesday,risk_country_High risk,risk_country_Low risk,risk_country_Medium risk,risk_country_Very High risk,age_category_ > 70,age_category_40 - 49,age_category_50 -59,age_category_60 - 69,age_category_< 40,period_of_the_day_arvo,period_of_the_day_early arvo,period_of_the_day_early morning,period_of_the_day_evening,period_of_the_day_late night,period_of_the_day_morning
151060,-0.869513,-0.255881,-0.231927,0.310352,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,1.0,1.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,1.0,0.0,0.0
119520,0.714171,-0.255881,-0.231927,0.310352,0.0,0.0,1.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,1.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,1.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0
45813,0.659561,-0.255881,-0.231927,0.310352,0.0,1.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,1.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,1.0
34353,0.714171,-0.255881,4.311699,-3.222148,0.0,0.0,0.0,1.0,0.0,1.0,0.0,0.0,0.0,0.0,1.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0
61518,0.932610,-0.255881,-0.231927,0.310352,0.0,0.0,0.0,1.0,0.0,1.0,0.0,0.0,0.0,1.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,1.0,0.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
51799,0.222682,3.766894,-0.231927,0.310352,0.0,0.0,0.0,1.0,0.0,0.0,1.0,0.0,0.0,0.0,1.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,1.0,0.0,0.0,0.0
69108,-1.306392,-0.255881,-0.231927,0.310352,0.0,1.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,1.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,1.0,0.0,0.0,0.0,0.0
131577,0.932610,-0.255881,-0.231927,0.310352,0.0,0.0,0.0,1.0,0.0,0.0,1.0,0.0,0.0,1.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,1.0,0.0,0.0
140342,1.096439,-0.255881,-0.231927,0.310352,0.0,0.0,0.0,1.0,0.0,1.0,0.0,0.0,0.0,0.0,1.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0


In [22]:
# label_encoders = {}
# for column in category_columns:
#     label_encoder = LabelEncoder()
#     data_train[column] = label_encoder.fit_transform(data_train[column])
#     label_encoders[column] = label_encoder  # Lưu trữ label encoder nếu cần sau này

# scaler = StandardScaler()
# data_train[numeric_columns] = scaler.fit_transform(data_train[numeric_columns])
# data_train

In [28]:
class_weights = compute_class_weight(class_weight='balanced', classes=np.unique(np.argmax(y_train, axis=1)), y=np.argmax(y_train, axis=1))
class_weights_dict = {i : class_weights[i] for i in range(len(class_weights))}
np.savez(f'{save_dir}/class_weights_dict.npz', class_weights_dict = class_weights_dict)
class_weights_dict

{0: 0.5518701828211362, 1: 5.319724674232871}

In [ ]:
from imblearn.over_sampling import RandomOverSampler
ros = RandomOverSampler(random_state=42)
X_rosampled, y_rosampled = ros.fit_resample(data_train.drop(columns=['class']), data_train['class'])
X_rosampled = pd.DataFrame(X_rosampled)
y_rosampled = pd.DataFrame(y_rosampled)
oversampling_sample=pd.concat([X_rosampled, y_rosampled], axis=1)    

In [ ]:
from sdv.metadata import SingleTableMetadata
metadata_sync = SingleTableMetadata()
metadata_sync.detect_from_dataframe(data=oversampling_sample)

In [ ]:
from sdv.single_table import GaussianCopulaSynthesizer



gau_synthesizer = GaussianCopulaSynthesizer(metadata_sync,
                                            enforce_min_max_values=True,  
                                            enforce_rounding=True,
                                            default_distribution='gamma') 

gau_synthesizer.fit(oversampling_sample)
gau_synthesizer.save(
    filepath=f'{save_dir}/gau_synthesizer.pkl'
)

In [ ]:
gau_synthetic_data = gau_synthesizer.sample(num_rows=len(data_train))
gau_synthetic_data.head()

In [ ]:
from sdv.evaluation.single_table import run_diagnostic



diagnostic_gau = run_diagnostic(
    real_data=oversampling_sample,
    synthetic_data=gau_synthetic_data,
    metadata=metadata_sync
)




In [ ]:
from sdv.evaluation.single_table import evaluate_quality



diagnostic_gau = evaluate_quality(
    real_data=oversampling_sample,
    synthetic_data=gau_synthetic_data,
    metadata=metadata_sync
)




In [ ]:
from sklearn.utils import shuffle
from imblearn.under_sampling import RandomUnderSampler

rus = RandomUnderSampler(random_state=42)

## Gau data
gau_data=gau_synthetic_data[gau_synthetic_data.class==1]
gau_sample=pd.concat([data_train, gau_data] , axis=0, ignore_index=True)
X_gausampled, y_gausampled = rus.fit_resample(gau_sample.drop(columns=['class']), gau_sample['class'])
X_gausampled = pd.DataFrame(X_gausampled)
y_gausampled = pd.DataFrame(y_gausampled)
gau_sample=pd.concat([X_gausampled, y_gausampled], axis=1)    
gau_sample = shuffle(gau_sample, random_state=42)
print(gau_sample.class.value_counts())

In [ ]:
gau_sample.to_excel(f'{save_dir}/data_train.xlsx', index=False)

In [ ]:
X_train = gau_sample.drop(columns=['class'])
y_train = gau_sample['class']

In [ ]:
import pandas as pd
import numpy as np
from sklearn.preprocessing import OneHotEncoder, StandardScaler


encoder = OneHotEncoder(handle_unknown="ignore", sparse_output=False)
X_train_cat = encoder.fit_transform(gau_sample[category_columns])
X_test_cat = encoder.transform(data_test[category_columns])  # Chỉ transform trên test

X_train_cat_df = pd.DataFrame(X_train_cat, columns=encoder.get_feature_names_out(category_columns))
X_test_cat_df = pd.DataFrame(X_test_cat, columns=encoder.get_feature_names_out(category_columns))

X_train_cat_df.index = gau_sample.index
X_test_cat_df.index = data_test.index

X_train = pd.concat([gau_sample[numeric_columns], X_train_cat_df], axis=1)
X_test = pd.concat([data_test[numeric_columns], X_test_cat_df], axis=1)

y_encoder = OneHotEncoder(sparse_output=False)
y_train = y_encoder.fit_transform(gau_sample[["class"]])
y_test = y_encoder.transform(data_test[["class"]])

scaler = StandardScaler()
X_train[numeric_columns] = scaler.fit_transform(X_train[numeric_columns])
X_test[numeric_columns] = scaler.transform(X_test[numeric_columns])  #

print(X_train.shape, X_test.shape)  
print(y_train.shape, y_test.shape) 

y_train = pd.DataFrame(y_train)
y_test = pd.DataFrame(y_test)

# Lưu X_train, X_test, y_train và y_test vào file CSV
X_train = pd.DataFrame(X_train)
X_test = pd.DataFrame(X_test)

In [ ]:

X_train.to_csv(f'{save_dir}/x_train.csv', index=False)
y_train.to_csv(f'{save_dir}/y_train.csv', index=False)

In [29]:

# for column in outliers_list:

X_test.to_csv(f'{save_dir}/x_test.csv', index=False)
y_test.to_csv(f'{save_dir}/y_test.csv', index=False)